In [ ]:
!pip install mlflow boto3 awscli optuna imbalanced-learn

In [ ]:
!aws configure

AWS Access Key ID [****************UHPB]: 
AWS Secret Access Key [****************7v2U]: 
Default region name [None]: 
Default output format [None]: 


In [ ]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-54-196-109-131.compute-1.amazonaws.com:5000/")

In [ ]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='s3://campusx-mlflow/608990915555109586', creation_time=1728589727081, experiment_id='608990915555109586', last_update_time=1728589727081, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


In [ ]:
df = pd.read_csv('/content/reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [ ]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Logistic Regression

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Logistic Regression
def objective_logreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    # LogisticRegression model setup with balanced class weight
    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Logistic Regression, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver='liblinear', random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LogisticRegression"
    log_mlflow("LogisticRegression", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Logistic Regression
run_optuna_experiment()


[I 2024-10-11 04:21:59,570] A new study created in memory with name: no-name-f8428230-2c76-41bd-83e9-593486875acc
[I 2024-10-11 04:22:00,431] Trial 0 finished with value: 0.7746776580004228 and parameters: {'C': 9.4566524398181, 'penalty': 'l2'}. Best is trial 0 with value: 0.7746776580004228.
[I 2024-10-11 04:22:01,045] Trial 1 finished with value: 0.7742549143944197 and parameters: {'C': 7.219097929737698, 'penalty': 'l2'}. Best is trial 0 with value: 0.7746776580004228.
[I 2024-10-11 04:22:01,303] Trial 2 finished with value: 0.7792221517649546 and parameters: {'C': 0.5670204003193772, 'penalty': 'l1'}. Best is trial 2 with value: 0.7792221517649546.
[I 2024-10-11 04:22:01,784] Trial 3 finished with value: 0.7688649334178821 and parameters: {'C': 2.5002578039429113, 'penalty': 'l2'}. Best is trial 2 with value: 0.7792221517649546.
[I 2024-10-11 04:22:01,944] Trial 4 finished with value: 0.6568378778270979 and parameters: {'C': 0.0038474664198131212, 'penalty': 'l2'}. Best is trial 2